In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/Fashion_mnist')

In [ ]:
!pip install vit-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 5.7 MB/s eta 0:00:00


In [ ]:
#You need to install the following python packages
#pytorch, vit_pytorch.
import torch
import torchvision
from vit_pytorch import ViT
import time
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(97)

In [ ]:
!pwd

/content


In [ ]:
Dpath = '/data/mnist'
Bs_Train = 100
Bs_Test = 1000

tform_mnist = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                               torchvision.transforms.RandomRotation(20),
                                               torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
                                                torchvision.transforms.RandomHorizontalFlip(),
                                                  torchvision.transforms.Normalize((0.1307,),(0.3081,))])

tr_set = torchvision.datasets.MNIST(Dpath, train = True, download = True,
                                       transform = tform_mnist)

tr_load = torch.utils.data.DataLoader(tr_set, batch_size = Bs_Train, shuffle = True)

ts_set = torchvision.datasets.MNIST(Dpath, train = False, download = True, transform = tform_mnist)

ts_load = torch.utils.data.DataLoader(ts_set, batch_size = Bs_Test, shuffle = True)

def train_iter(model, optimz, data_load, loss_val):
    samples = len(data_load.dataset)
    model.train()
    
    for i, (data, target) in enumerate(data_load):
        optimz.zero_grad()
        out = F.log_softmax(model(data), dim=1)
        loss = F.nll_loss(out, target)
        loss.backward()
        optimz.step()
        
        if i % 100 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_load)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_val.append(loss.item())


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



In [ ]:
a = next(iter(tr_load))
a[0].size()

torch.Size([100, 1, 28, 28])

In [ ]:
def evaluate(model, data_load, loss_val):
    model.eval()
    
    samples = len(data_load.dataset)
    csamp = 0
    tloss = 0

    with torch.no_grad():
        for data, target in data_load:
            output = F.log_softmax(model(data), dim=1)
            loss = F.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            tloss += loss.item()
            csamp += pred.eq(target).sum()

    aloss = tloss / samples
    loss_val.append(aloss)
    print('\nAverage test loss: ' + '{:.4f}'.format(aloss) +
          '  Accuracy:' + '{:5}'.format(csamp) + '/' +
          '{:5}'.format(samples) + ' (' +
          '{:4.2f}'.format(100.0 * csamp / samples) + '%)\n')
    
N_EPOCHS = 10

start_time = time.time()
model = ViT(image_size=28, patch_size=4, num_classes=10, channels=1,
            dim=32, depth=8, heads=4, mlp_dim=64)
optimz = optim.Adam(model.parameters(), lr=0.003)

trloss_val, tsloss_val = [], []
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, tr_load, trloss_val)
    evaluate(model, ts_load, tsloss_val)

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/60000 (  0%)]  Loss: 2.3645
[10000/60000 ( 17%)]  Loss: 0.9707
[20000/60000 ( 33%)]  Loss: 0.6061
[30000/60000 ( 50%)]  Loss: 0.5027
[40000/60000 ( 67%)]  Loss: 0.4981
[50000/60000 ( 83%)]  Loss: 0.5070

Average test loss: 0.4050  Accuracy: 8660/10000 (86.60%)

Epoch: 2
[    0/60000 (  0%)]  Loss: 0.4129
[10000/60000 ( 17%)]  Loss: 0.2687
[20000/60000 ( 33%)]  Loss: 0.6565
[30000/60000 ( 50%)]  Loss: 0.4998
[40000/60000 ( 67%)]  Loss: 0.3387
[50000/60000 ( 83%)]  Loss: 0.3196

Average test loss: 0.3166  Accuracy: 8944/10000 (89.44%)

Epoch: 3
[    0/60000 (  0%)]  Loss: 0.2989
[10000/60000 ( 17%)]  Loss: 0.2298
[20000/60000 ( 33%)]  Loss: 0.3183
[30000/60000 ( 50%)]  Loss: 0.3896
[40000/60000 ( 67%)]  Loss: 0.3109
[50000/60000 ( 83%)]  Loss: 0.3334

Average test loss: 0.2309  Accuracy: 9239/10000 (92.39%)

Epoch: 4
[    0/60000 (  0%)]  Loss: 0.3130
[10000/60000 ( 17%)]  Loss: 0.3114
[20000/60000 ( 33%)]  Loss: 0.3680
[30000/60000 ( 50%)]  Loss: 0.2947
[40000/60000 ( 67